In [31]:
deck_columns = ['gameMode_name']+[f'team_0_cards_{x}_name' for x in range(8)]+[f'opponent_0_cards_{x}_name' for x in range(8)]+['team_0_crowns']+['opponent_0_crowns']

In [39]:
import os
import pandas as pd

df_match = pd.DataFrame()
for csv_file in os.listdir('CRL_matches'):
    df = pd.read_csv('CRL_matches/'+csv_file)
    #display(df[deck_columns])
    df_match = pd.concat([df_match, df[deck_columns]])
df_match.to_csv('CRL_matches.csv')

## EDA

In [195]:
import pandas as pd
import numpy as np

df = pd.read_csv('CRL_1_raw.csv').drop('Unnamed: 0', axis=1)
df.gameMode_name.unique()

array(['Ladder', 'Friendly', 'RAMRIDER_Rage_Buff_Tournament',
       'Draft_Competitive', 'Challenge_AllCards_EventDeck', 'TeamVsTeam',
       'TripleElixir_Ladder', 'MirrorDeck_Friendly', 'Showdown_Ladder',
       'ClanWar_BoatBattle', 'TeamVsTeamLadder', 'Ladder_CrownRush',
       'TeamVsTeam_TripleElixir_Friendly', 'Touchdown_Draft',
       'CW_Battle_1v1', 'Ladder_GoldRush', 'DoubleDeck_Tournament',
       'CW_Duel_1v1', 'Tournament', 'DoubleElixir_AllCards_EventDeck',
       '7xElixir_Friendly', 'Heist_Friendly', 'Overtime_Tournament',
       'TripleElixir_Friendly', 'RampUpElixir_Friendly', 'DraftMode',
       'TripleElixir_Tournament', 'ClassicDecks_Friendly',
       'Friendly_FixedDeckOrder', 'Event_ArcherQueen_Tease',
       'Event_SkeletonKing_Tease', 'Touchdown',
       'Overtime_AllCards_EventDeck', 'TeamVsTeam_Touchdown_Draft',
       'Overtime_Friendly', 'DoubleElixir_Friendly', 'Rage_Friendly',
       'DraftModeInsane'], dtype=object)

In [198]:
#only select proper game modes
df_clean = df[(df['gameMode_name']=='Ladder') | (df['gameMode_name']=='Friendly') | (df['gameMode_name']=='Overtime_Friendly') | (df['gameMode_name']=='Tournament') | (df['gameMode_name']=='Ladder_CrownRush') | (df['gameMode_name']=='Ladder_GoldRush') | (df['gameMode_name']=='CW_Battle_1v1') | (df['gameMode_name']=='CW_Duel_1v1')]

#only select decks, crowns are irrelevant
df_1 = df_clean[[
    'team_0_cards_0_name', 
    'team_0_cards_1_name', 
    'team_0_cards_2_name', 
    'team_0_cards_3_name', 
    'team_0_cards_4_name', 
    'team_0_cards_5_name', 
    'team_0_cards_6_name', 
    'team_0_cards_7_name']]
df_1.columns = list(range(8))

df_2 = df_clean[[
    'opponent_0_cards_0_name', 
    'opponent_0_cards_1_name',
    'opponent_0_cards_2_name', 
    'opponent_0_cards_3_name',
    'opponent_0_cards_4_name', 
    'opponent_0_cards_5_name',
    'opponent_0_cards_6_name', 
    'opponent_0_cards_7_name']]
df_2.columns = list(range(8))

#put everything together
df_clean = pd.concat([df_1, df_2], axis=0)
df_clean

,0,1,2,3,4,5,6,7
0,Ram Rider,Inferno Dragon,Lumberjack,Fireball,Bandit,Magic Archer,Barbarian Barrel,Ice Golem
1,Ram Rider,Inferno Dragon,Lumberjack,Fireball,Ice Golem,Magic Archer,Barbarian Barrel,Tombstone
2,Ram Rider,Inferno Dragon,Lumberjack,Fireball,Bandit,Magic Archer,Barbarian Barrel,Ice Golem
3,Ram Rider,Inferno Dragon,Lumberjack,Fireball,Bandit,Magic Archer,Barbarian Barrel,Ice Golem
4,Ram Rider,Inferno Dragon,Lumberjack,Fireball,Bandit,Magic Archer,Barbarian Barrel,Ice Golem
...,...,...,...,...,...,...,...,...
16563,Elite Barbarians,Elixir Collector,Bandit,Heal Spirit,Archer Queen,Royal Ghost,Barbarian Barrel,Three Musketeers
16564,Electro Giant,Bowler,Musketeer,Goblin Cage,Skeletons,Barbarian Barrel,Tornado,Lightning
16565,Ice Golem,Hog Rider,Fireball,Ice Spirit,Musketeer,The Log,Cannon,Skeletons
16566,P.E.K.K.A,Electro Wizard,Bandit,Battle Ram,Dark Prince,Fireball,Zap,Magic Archer


In [271]:
#index-based encoding
from sklearn import preprocessing
codex = preprocessing.LabelEncoder()
codex = codex.fit(pd.unique(df_clean.values.ravel('K')))
codex.transform(['Battle Healer'])
#codex.inverse_transform([10])

df_index = df_clean.apply(lambda x : codex.transform(x))
df_index

,0,1,2,3,4,5,6,7
0,79,56,61,30,5,62,6,53
1,79,56,61,30,53,62,6,97
2,79,56,61,30,5,62,6,53
3,79,56,61,30,5,62,6,53
4,79,56,61,30,5,62,6,53
...,...,...,...,...,...,...,...,...
16563,25,26,5,50,0,83,6,96
16564,22,14,72,40,91,6,98,60
16565,53,51,30,54,72,95,15,91
16566,74,24,5,11,18,30,104,62


In [273]:
#one_hot encoding
row_list = list()
#len(pd.unique(df_index.values.ravel('K')))
for row in range(df_index.shape[0]):
    a = np.array(df_index.iloc[row])
    b = np.zeros((a.size, 106))
    b[np.arange(a.size),a] = 1
    row_list.append(np.sum(b, axis=0))

df_onehot = pd.DataFrame(row_list) #.astype(int)
df_onehot.columns = [codex.inverse_transform([x])[0] for x in list(df_onehot.columns)]
df_onehot

,Archer Queen,Archers,Arrows,Baby Dragon,Balloon,Bandit,Barbarian Barrel,Barbarian Hut,Barbarians,Bats,...,Three Musketeers,Tombstone,Tornado,Valkyrie,Wall Breakers,Witch,Wizard,X-Bow,Zap,Zappies
0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23671,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23672,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23674,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [274]:
#implode horizontally
df_onehot_compressed = pd.DataFrame([[df_onehot.iloc[x].values] for x in range(len(df_onehot))])
df_onehot_compressed.index = df_onehot_compressed.index.set_names(['_id'])
df_onehot_compressed.reset_index()
df_onehot_compressed.columns = ['deck']
df_onehot_compressed

,deck
_id,
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ..."
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ..."
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ..."
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ..."
...,...
23671,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ..."
23672,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
23673,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [275]:
df_clean.to_csv('CRL_2_final.csv')

In [ ]:
#prepare for upload
df_ready = df_onehot_compressed.copy()
df_ready.index = df_ready.index.set_names(['_id'])
df_ready = df_ready.reset_index()
df_ready['deck'] = df_ready['deck'].apply(lambda x: list(x))
df_ready.columns = ['_id', 'deck_vector_']
df_ready = df_ready.to_dict(orient='records')
df_ready

In [ ]:
df_ready

## clustering

In [288]:
#   SDK
import relevanceai

project = "michelangioloma"
api_key = "TDFvdm9Yd0J1M3VVTEcyQTV3VWs6b3RnRXJnUW9SQ2kwMVVJWVZ2VzVtZw"
client = relevanceai.Client(project, api_key)
client.base_url = "https://gateway-api-aueast.relevance.ai/latest"

In [295]:
#client.datasets.delete(dataset_id='crl_matches')

{'status': 'complete', 'message': 'crl_matches deleted'}

In [ ]:
client.datasets.list()

In [297]:
#upload df
client.datasets.bulk_insert(dataset_id='crl_matches', documents=df_ready)

{'inserted': 23676, 'failed_documents': []}

In [299]:
client.datasets.schema('crl_matches')

{'deck_vector_': {'vector': 106}, 'insert_date_': 'date'}

In [300]:
# Vector field based on which clustering is done - (Currently only one vector is supported)
vector_field = 'descriptiontextmulti_vector_'

#local or remote?
centroids = client.vector_tools.cluster.kmeans_cluster(
    dataset_id = 'crl_matches', 
    vector_fields=['deck_vector_'],
    k = 6)

#creates clusters but only gives the centroids
#clustering results is uploaded on the database

100%|██████████| 24/24 [00:55<00:00,  2.31s/it]


Initialization complete
Iteration 0, inertia 242428.0
Iteration 1, inertia 134379.93934088608
Iteration 2, inertia 130668.76565957109
Iteration 3, inertia 130316.43468946217
Iteration 4, inertia 130282.5908531319
Iteration 5, inertia 130281.33682854341
Iteration 6, inertia 130281.2942457566
Converged at iteration 6: center shift 4.291710015118029e-06 within tolerance 6.6045644111417256e-06.
Initialization complete
Iteration 0, inertia 225810.0
Iteration 1, inertia 132835.937234555
Iteration 2, inertia 129236.3187987102
Iteration 3, inertia 128579.82946053825
Iteration 4, inertia 128552.27023944426
Iteration 5, inertia 128549.13892754483
Iteration 6, inertia 128542.15903706459
Iteration 7, inertia 128540.17253318666
Iteration 8, inertia 128527.98432634036
Iteration 9, inertia 128527.1075246823
Iteration 10, inertia 128522.64691616042
Iteration 11, inertia 128522.08896549275
Iteration 12, inertia 128522.06177946828
Converged at iteration 12: strict convergence.
Initialization complete
It

In [301]:
client.datasets.schema('crl_matches')

{'_cluster_': 'dict',
 '_cluster_.deck_vector_': 'dict',
 '_cluster_.deck_vector_.kmeans_6': 'text',
 'deck_vector_': {'vector': 106},
 'insert_date_': 'date'}

In [302]:
client.services.cluster.centroids.list_closest_to_center(
  dataset_id='crl_matches',
  vector_fields=["deck_vector_"],
  #cluster_ids=[], # Leave this as an empty list if you want all of the clusters
  alias="kmeans_6" #change to 'kmeans_10' 
)

You can now visit the dashboard at https://cloud.relevance.ai/sdk/cluster/centroids/closest


{'Cluster-0': [{'_id': '17166',
   'insert_date_': '2022-01-05T07:53:03.856Z',
   '_cluster_': {},
   '_search_score': 0.7341913}],
 'Cluster-1': [{'_id': '23482',
   'insert_date_': '2022-01-05T07:53:25.434Z',
   '_cluster_': {},
   '_search_score': 0.7237353}],
 'Cluster-2': [{'_id': '19710',
   'insert_date_': '2022-01-05T07:53:12.085Z',
   '_cluster_': {},
   '_search_score': 0.8830644000000001}],
 'Cluster-3': [{'_id': '14217',
   'insert_date_': '2022-01-05T07:52:53.592Z',
   '_cluster_': {},
   '_search_score': 0.7792193000000001}],
 'Cluster-4': [{'_id': '23128',
   'insert_date_': '2022-01-05T07:53:24.430Z',
   '_cluster_': {},
   '_search_score': 0.7465310999999999}],
 'Cluster-5': [{'_id': '12455',
   'insert_date_': '2022-01-05T07:52:48.478Z',
   '_cluster_': {},
   '_search_score': 0.8380650999999999}]}